In [1]:
import numpy as np
from util.data_preparation import VariedSizedImagesCollate, LoadingImgs_to_ListOfListOfTensors, VariedSizedImagesDataset, data_aug_preprocessing_HR, patch_wise_predict
import time
import torch
import torchvision.transforms as T
from options.train_options import TrainOptions
from models import create_model
def linear_normalize(tmp):
    return (tmp - tmp.min())/(tmp.max() - tmp.min())
import matplotlib.pyplot as plt
import os
from PIL import Image
import imageio

In [2]:
'''load data'''
data_folder = '../Data/Data_0309_23/'

list_of_list_of_tensors = []
num_imgs = 35
for i in range(1, num_imgs+1):
    # X = 1 - np.array(Image.open(data_folder + 'OCT_'+str(i)+'.tif')).astype('float32')[np.newaxis, np.newaxis, :, :]/255.0
    # Y = np.array(Image.open(data_folder + 'GS_stain_normalized_'+str(i)+'.tif')).astype('float32')[np.newaxis,:,:,:]/255.0
    X = 1.0 - np.load(data_folder+'x_mus_'+str(i)+'.npy').astype('float32')[np.newaxis, np.newaxis, :, :]
    Y = np.load(data_folder+'y_gallyas_'+str(i)+'.npy').astype('float32')[np.newaxis,:,:,:]
    list_of_list_of_tensors.append([torch.Tensor(X), torch.Tensor(np.transpose(Y, [0,3,1,2]))])
dataset = VariedSizedImagesDataset(list_of_list_of_tensors)
## batch_size can only be 1
dataloader = torch.utils.data.DataLoader(dataset,batch_size=1, collate_fn=VariedSizedImagesCollate, shuffle=False,
                                         num_workers=0, drop_last=True)

In [9]:
'''create model and load model weights'''
model = '--model cutreg_twostage'
opt = TrainOptions(model).parse()   # get training options
opt.input_nc = 3
model = create_model(opt)      # create a model given opt.model and other options

'''CUT model weights'''
# G_weights_name = 'adam_lin_GAN1_NCEX20_NCEY10_lr0.0003_apr26_CUT/190_net_G'

'''FastCUT model weights'''
# G_weights_name = 'CUT_9spl_4x_GAN1_NCE20_3Channel/280_net_G'

'''Pseudo_supervised model weights'''
# G_weights_name = 'PseudoSupervised_9spl_4x_MAEpseudo10_3channel/270_net_G'

'''Pseudo + CUT + MAE_reg model weights'''
# G_weights_name = 'CUT+REG+Pseudo_9spl_4x_NCE20_REG100_TV600_MAE1_MAEpseudo10_3channel/260_net_G'

'''ablation study'''
'''Pseudo + CUT model weights'''
# G_weights_name = 'CUT+Pseudo_9spl_4x_NCE20_MAEpseudo10_3channel/140_net_G'

'''CUT + REG model weights'''
# G_weights_name = 'CUT+REG_9spl_4x_NCE20_REG100_TV600_MAE1_3channel/200_net_G'

'''Registration only'''
G_weights_name = 'PreREG_9spl_4x_REG100_TV600_MAE10_3channel/260_net_G'

'''Pseudo label only'''
# G_weights_name = 'PseudoSupervised_9spl_4x_MAEpseudo10_3channel/300_net_G'

----------------- Options ---------------
                 CUT_mode: FastCUT                       
                LAB_space: False                         
               batch_size: 1                             
                    beta1: 0.5                           
                    beta2: 0.999                         
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: placeholder                   
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: None                          
            display_ncols: 4                             
             display_port: 8097                          
           display_server: htt

'Pseudo label only'

In [10]:
model.netG.load_state_dict(torch.load('checkpoints/'+G_weights_name+'.pth'))
test_list_numpy = []
gt_list_numpy = []

img_dir = '../Data/Test_results/' + G_weights_name + '/processed/'
if not os.path.exists(img_dir):
    os.makedirs(img_dir)
    print(f'Directory {img_dir} created')
else:
    print(f'Directory {img_dir} already exists')

for i, data in enumerate(dataloader):
    model.set_input(data)
    fake_B = model.patch_wise_predict(input_data=model.real_A_eq, bs=4, stride_ratio=4).cpu().numpy()
#     imageio.imwrite(img_dir + 'translated_.tif', (fake_B*255).astype(np.uint8))
    Image.fromarray((fake_B*255).astype(np.uint8)).save(img_dir + 'translated_' + str(i + 1) + '.png')
    np.save(img_dir + 'translated_' + str(i + 1) + '.npy', fake_B)

Directory ../Data/Test_results/PreREG_9spl_4x_REG100_TV600_MAE10_3channel/260_net_G/processed/ created
